# -Introduction
l'agence de stante publique France a lance un apel a projets pour trouver
des idees innovantes d'applicationq en lien avec l'alimentation
Nous avons decidé de nous attaquer a l'obésite.
Nous voulons etudier la possiblite qu'une application informe, pour n'importe quel produit,
le consommation du caractere Nutriscore.
La prediction du Nutriscore doit etre realise a partir de la liste des nutiments du produit.
Le Nutriscore est une classification des produits alimentaires, les classes sont nommées par ordre
de qualite nutritionnelle de A a E


## Hypothese principal :
Est il possibel de predire le Nutriscore a partire de la declaration nutitionnelle obligatoire du produit?
La declaration nutritionnelle suivant le reglement INCO

## Hypothese secondaires :
Pouvons nous classer nos produits en 3 categories: {divers,matieresgrasses ajoutees, boissons}
a partir de la declaration nutritionnelle obligatoire du produit? a minima a a partir d'une liste etendue de nutriments?

## demarche de resolution:
Nous allons devoir resoudre un problem de classification, et donc pour repiondre a ces questions, nous devons analyser notre jeu de donnes pour determiner si les nutriments agissent sur la classe du nutriscore.
Pour cela nous allons devoir nous concentrer sur les caracteristiques essentiels:

#### Nom des colonnes = [code_barres, nutriscore_grade, energie_100g, matieres_grasses_100g, matieres_grasses_saturees_100g, glucides_100g, glucides_sucres_100g, protein_100g, sel_100g]
# jeu de donnees:
Open Food FACTS.
    * les information generales sur la fiche du produit: nom, date de modification, etc.
    * un ensemble de tags: categorie du produit, localisation, origine, etc.
    * les ingredients composant les produits et leurs additifs eventuels.
    * des informatrion nutritionnelles: quantite en grammes d'un nutriment pour 100 grammes du produit.

# Librairies : 
    * Numpy permet de manipuler des tableaux de nombres uniquement ainsi que des fonctions mathematiqueq operant sur ces tableaux
    * pandas etend les possiblites de numpy en permettant de manipuler facilement des tableaux de données mixtes.
    * seaborn permet de realiser des graphique et des statistique sir des dataframes de pandas.

# Fonctions Helper:
Nous allons utiliser ces fonctions d'aides afin de reduire le temps de nettoyage et analyse:
    * percentage_nan_by_columns().
    * get_colors_from_listedcolormap().
    * get_set_of_labels().
    * plot_sghisto().
    * do_krustal_test().

# Collection des donnees :
##### Quelle source de donees existe-t-il?
il est demande expressement d'utiliser les donnees provenant d'open food Facts.
les champs sont separers en quatre section:
    * Les informations generales sur la fiche du produit: nom, date de modification, etc?
    * Un ensemble de tags: categories du produit, localisation, origine, etc.
    * Les ingredients composant les produits et leur additifs eventuels
    * Les informations nutritionnelle: quantite en grammes d'un nutriment pour 100 grammes du produit.
##### Quelle est la reglementation sur la protection de ces donnees?
aucaun , la source de donnees est publique.
##### ou sont stockees les donnees?
elles sont sur les serveurs distants publiques et sur notre ordinateur en sdd local.
##### Quel est le type des donnees :
structure




# Importation des données 

In [1]:
import dask.dataframe as dd
import numpy as np
import pandas as pd
import re
import IPython.display
import matplotlib.pyplot as plt 
import seaborn as sns
plt.style.use("seaborn")
%matplotlib inline
dtypes = {
    "brand_owner":object,
    "generic_name":object,
    "ingredients_from_palm_oil_tags":object,
    "ingredients_that_may_be_from_palm_oil_tags":object,
    "origins":object,
    "origins_en":object,
    "origins_tags":object,
    "packaging_text":object,
    "traces":object,
    "traces_en":object,
    "traces_tags":object,
    "code":object,
    "abbreviated_product_name": object
    }
#data = dd.read_csv("data.csv", sep = "\t", dtype=dtypes, low_memory=False).compute()


In [31]:
url = "D://data food//en.openfoodfacts.org.products.csv"
df = pd.read_csv(url, sep=',', header=None,error_bad_lines=False)

b'Skipping line 26: expected 1 fields, saw 52\nSkipping line 93: expected 1 fields, saw 52\nSkipping line 94: expected 1 fields, saw 52\nSkipping line 120: expected 1 fields, saw 52\nSkipping line 133: expected 1 fields, saw 104\nSkipping line 210: expected 1 fields, saw 75\nSkipping line 304: expected 1 fields, saw 57\nSkipping line 392: expected 1 fields, saw 54\nSkipping line 484: expected 1 fields, saw 71\nSkipping line 505: expected 1 fields, saw 87\nSkipping line 519: expected 1 fields, saw 157\nSkipping line 529: expected 1 fields, saw 73\nSkipping line 592: expected 1 fields, saw 120\nSkipping line 654: expected 1 fields, saw 75\nSkipping line 665: expected 1 fields, saw 67\nSkipping line 910: expected 1 fields, saw 60\nSkipping line 985: expected 1 fields, saw 96\nSkipping line 1074: expected 1 fields, saw 87\nSkipping line 1138: expected 1 fields, saw 91\nSkipping line 1368: expected 1 fields, saw 78\nSkipping line 1629: expected 1 fields, saw 52\nSkipping line 1791: expected

In [11]:
## Afficher row of data from url
df.head()

,code_barres,date_creation,nom_produit,categories,countries,nutriscore_score,nutriscore_grade,pnns_groupes_1,pnns_groupes_2,energie_100g,matieres_grasses_100g,matieres_grasses_saturees_100g,glucides_100g,glucides_sucres_100g,fibres_100g,proteines_100g,sel_100g,sodium_100g
0,100,1970-01-01 00:00:01.444572561,moutarde au moût de raisin,"Epicerie, Condiments, Sauces, Moutardes",France,18.0,d,Fat and sauces,Dressings and sauces,936.0,8.2,2.2,29.00,22.00,0.0,5.1,4.600,1.8400
1,1281,1970-01-01 00:00:01.517830801,Tarte noix de coco,"Tartes, Tartes sucrées, Tartes à la noix de coco",France,14.0,d,Sugary snacks,Biscuits and cakes,1594.0,22.0,15.5,27.30,21.90,4.4,4.6,0.100,0.0400
2,1885,1970-01-01 00:00:01.511180337,Compote de poire,"Aliments et boissons à base de végétaux, Alime...",France,-2.0,a,Fruits and vegetables,Fruits,657.0,0.0,0.0,36.00,27.00,3.6,0.6,0.000,0.0000
3,5470,1970-01-01 00:00:01.511778860,BAguette bressan,"Aliments et boissons à base de végétaux, Alime...",France,-4.0,a,Cereals and potatoes,Bread,669.0,2.2,0.5,25.20,0.60,1.6,9.5,0.358,0.1432
4,43595,1970-01-01 00:00:01.489050424,Cranberries,"Plant-based foods and beverages, Plant-based f...",United States,3.0,c,Fruits and vegetables,Fruits,1255.0,0.0,0.0,83.33,66.67,10.0,0.0,0.000,0.0000


Le chargment des donness retourne un evertissement sur la presence de tye different dans la colonne 0
le code a barre est realite une chaine de caractere, car certains sont trop longs pour etre stoches en type dloat 64 par le syteme


# Import des fonctions

In [6]:
# Retourne une serie comportant le pourcentage de valeurs manquantes pour chaques conlonnes du plateforme

def percentage_nan_by_columns(df):
    s = df.isnull().sum(axis=0) / dataframe.shap[0] * 100
    s = s.transform(lambda x:'{:02.2f}'.format(x) + '%')
    return s


In [33]:
# prendre connaissance du veritable contenu et de la qualite de mes donnese.
df.info(verbose= True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030575 entries, 0 to 1030574
Data columns (total 1 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   0       1030575 non-null  object
dtypes: object(1)
memory usage: 7.9+ MB


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030575 entries, 0 to 1030574
Data columns (total 1 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   0       1030575 non-null  object
dtypes: object(1)
memory usage: 7.9+ MB
